# <b> Part 2. After loading a NEEM, you can analyse it for the contained parameters<b>
# In this Notebook you will access a food cutting knowledge graph to retrieve task information

The idea is that you first loaded an episodic memory/ experiment for inspection and now want to retrieve additional parameters. If you did not select an experiment, you can <b>set your parameters<b> here

In [14]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
cuts = [('1',"owl:hasValue"),
        ('> 1', "owl:minQualifiedCardinality")]

poses=[('center', "cut:halving_position"),
        ('end', "cut:slicing_position")]

rotations=[('<= 1', "single"),
        ('> 1', "multiple")]

cut=""
pose=""
rotation=""

def chooseCuts(Cut):
    global cut
    cut=Cut
     
def choosePose(Pose):
    global pose
    pose=Pose
    
def chooseRotation(Rotation):
    global rotation
    rotation=Rotation

cut_widget = widgets.Dropdown(options=cuts, description='No. of Cuts:')
pose_widget = widgets.Dropdown(options=poses, description='Cutting Pose:')
rotation_widget = widgets.Dropdown(options=rotations, description='No. of Rotations:')

def cutEvent(event):
    chooseCuts(event.new)

def poseEvent(event):
    choosePose(event.new)

def rotateEvent(event):
    chooseRotation(event.new)
    
cut_widget.observe(cutEvent, names='value')
pose_widget.observe(poseEvent, names='value')
rotation_widget.observe(rotateEvent, names='value')

widgets_display = HBox([cut_widget, pose_widget, rotation_widget])

widgets_display

# optional: set Parameters manually:
#cuts = ""
#pose = ""
#rotation = ""

### We need to also install some libraries

In [15]:
!pip install rdflib
# import rdflib
from rdflib import Graph, Literal, URIRef, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

## With the given parameters, we can infer the task that was performed in the demonstration

In [27]:
g = Graph()

# define prefixes to be used in the query 
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT = Namespace("http://www.ease-crc.org/ont/food_cutting#")

g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("cut", CUT)


# the actual query comes here
query = """SELECT ?task WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {
       ?s %s ?o.
       ?task rdfs:subClassOf ?s.
       ?task rdfs:subClassOf ?no.
       ?no owl:someValuesFrom %s.
   FILTER (?task != cut:Julienning && ?task != soma:Dicing && ?task != cut:Quartering)} }""" % (cut,pose) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects
    
if rotation == "multiple": #for simplicity we define all tasks that involve multiple turning movements as dicing tasks, since we need additional info to be more specific
    task = "http://www.ease-crc.org/ont/food_cutting#Dicing"
    print(task)
else: 
    for row in g.query(query):
        print(row.task)



http://www.ease-crc.org/ont/food_cutting#Dicing
